# Differentiable Physics Application

Source: https://scipython.com/blog/the-brachistochrone-problem/

# Approximating the Brachistochrone Curve with a Neural Network

## Problem Statement

I want to approximate the curve \( y = f(x) \) using a neural network. To do so, I need a physics-based model to define my loss function. The loss will be the total time taken by a ball to travel along the curve under gravity, and I will minimize this time.

## Physics Model

The time taken for the ball to travel along the curve is given by the integral:

$$ 
T = \int_0^L \frac{ds}{v} 
$$

where:

- \( L \) is the total arc length of the curve.
- \( ds \) is the differential arc length.
- \( v \) is the velocity of the ball.

---

### Conservation of Energy

The ball must obey the conservation of energy:

$$ 
\frac{1}{2} m v^2 = mg y 
$$

where:

- \( m \) is the mass of the ball.
- \( v \) is its velocity.
- \( g = 9.81 \, m/s^2 \) is the gravitational acceleration.



Rearranging for \( v \):

$$ 
v = \sqrt{2 g y} 
$$

Substituting this into the time integral:

$$ 
T = \int_0^L \frac{ds}{\sqrt{2 g y}} 
$$

### Arc Length Differential

From differential geometry, the arc length element \( ds \) satisfies:

$$ 
ds^2 = dx^2 + dy^2 
$$

**Why?** This comes from the Pythagorean theorem applied to an infinitesimal segment of the curve.

Thus, we substitute:

$$ 
ds = \sqrt{dx^2 + dy^2} 
$$

So the final integral for time becomes:

$$ 
T = \int_0^L \frac{\sqrt{dx^2 + dy^2}}{\sqrt{2 g y}} 
$$

This integral will serve as the loss function for training the neural network.
